In [21]:
import requests
import pandas as pd
from datetime import datetime, timedelta

NBA_SCORES_URL = "https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard"

def get_nba_results():
    yesterday = datetime.now() - timedelta(days=1)
    response = requests.get(f"{NBA_SCORES_URL}?dates={yesterday.strftime('%Y%m%d')}")
    if response.status_code != 200:
        print("Failed to retrieve data")
        return None
    
    data = response.json()
    games = data.get("events", [])
    
    results = []
    for game in games:
        game_date = game.get("date", "N/A")
        if game_date != "N/A":
            game_datetime = datetime.strptime(game_date, "%Y-%m-%dT%H:%MZ") - timedelta(hours=5)
            readable_date = game_datetime.strftime("%B %d, %Y")
        else:
            readable_date = "N/A"
        
        home_team = game["competitions"][0]["competitors"][0]["team"]["displayName"]
        home_score = game["competitions"][0]["competitors"][0].get("score", "N/A")
        away_team = game["competitions"][0]["competitors"][1]["team"]["displayName"]
        away_score = game["competitions"][0]["competitors"][1].get("score", "N/A")
        
        winner = None
        if home_score != "N/A" and away_score != "N/A":
            if int(home_score) > int(away_score):
                winner = home_team
            elif int(away_score) > int(home_score):
                winner = away_team
        
        game_info = {
            "Date": readable_date,
            "Home Team": home_team,
            "Home Score": home_score,
            "Away Team": away_team,
            "Away Score": away_score,
            "Winner": winner
        }
        results.append(game_info)
    
    return pd.DataFrame(results)

def highlight_winner(row):
    styles = [''] * len(row)
    
    if row['Home Score'] != "N/A" and row['Away Score'] != "N/A":
        if int(row['Home Score']) > int(row['Away Score']):
            styles[row.index.get_loc('Home Team')] = 'background-color: green'
            styles[row.index.get_loc('Home Score')] = 'background-color: green'
        elif int(row['Away Score']) > int(row['Home Score']):
            styles[row.index.get_loc('Away Team')] = 'background-color: green'
            styles[row.index.get_loc('Away Score')] = 'background-color: green'
    
    return styles

df = get_nba_results()

df = df.drop(columns=['Winner'])

df.style.apply(highlight_winner, axis=1)


,Date,Home Team,Home Score,Away Team,Away Score
0,"March 20, 2025",Charlotte Hornets,115,New York Knicks,98
1,"March 20, 2025",Indiana Pacers,105,Brooklyn Nets,99
2,"March 20, 2025",Golden State Warriors,117,Toronto Raptors,114
3,"March 20, 2025",Sacramento Kings,116,Chicago Bulls,128
4,"March 20, 2025",Los Angeles Lakers,89,Milwaukee Bucks,118
